# Chapter 18. Metric Predicted Variable with Multiple Metric Predictors

# Contents

* 18.1. Multiple Linear Regression
* 18.2. Multiplicative Interaction of Metric Predictors
* 18.3. Shrinkage of Regression Coefficients
* 18.4. Variable Selection

In this chapter, we are concerned with situations in which the value to be predicted is on a metric scale, and there are several predictors, each of which is also on a metric scale. 

* For example, 
    - we might predict 
        - a person’s college grade point average (GPA) from 
            - his or her high-school GPA and 
            - scholastic aptitude test (SAT) score. 
    - Another such situation is predicting 
        - a person’s blood pressure from 
            - his or her height and 
            - weight.

* multiple linear regression 
    - predicted variable is an additive combination of predictors
* interactions 
    - nonadditive combinations of predictors

<img src="figures/tbl15.1.png" width=600 />
<img src="figures/tbl15.2.png" width=600 />
<img src="figures/tbl15.3.png" width=600 />

# 18.1. Multiple Linear Regression

* 18.1.1 The perils of correlated predictors
* 18.1.2 The model and implementation
* 18.1.3 The posterior distribution
* 18.1.4 Redundant predictors
* 18.1.5 Informative priors, sparse data, and correlated predictors

y ∼ normal(μ, σ ) and μ = β0+β1x1+β2x2.

## 18.1.1 The perils of correlated predictors

<img src="figures/fig18.1.png" width=600 />

<img src="figures/fig18.2.png" width=600 />

<img src="figures/fig18.3.png" width=600 />

## 18.1.2 The model and implementation

<img src="figures/fig18.4.png" width=600 />

The data block of the JAGS code then standardizes the predictors by looping
through the columns of the x matrix, as follows:

In [ ]:
data {
    ym <- mean(y)
    ysd <- sd(y)
    for ( i in 1:Ntotal ) { # Ntotal is the number of data rows
        zy[i] <- ( y[i] - ym ) / ysd
    }
    for ( j in 1:Nx ) {# Nx is the number of x predictors
        xm[j] <- mean(x[,j]) # x is a matrix, each column a predictor
        xsd[j] <- sd(x[,j])
        for ( i in 1:Ntotal ) {
        zx[i,j] <- ( x[i,j] - xm[j] ) / xsd[j]
        }
    }
}

The standardized parameters are then transformed to the original scale by generalizing Equation 17.2 (p. 485) to multiple predictors:


<img src="figures/eq18.1.png" width=600 />

The JAGS model specification looks like this:

In [ ]:
model {
    for ( i in 1:Ntotal ) {
        zy[i]  ̃ dt( zbeta0 + sum( zbeta[1:Nx] * zx[i,1:Nx] ) , 1/zsigmaˆ2 , nu )
    }
    # Priors vague on standardized scale:
    zbeta0  ̃ dnorm( 0 , 1/2ˆ2 )
    for ( j in 1:Nx ) {
        zbeta[j]  ̃ dnorm( 0 , 1/2ˆ2 )
    }
    zsigma  ̃ dunif( 1.0E-5 , 1.0E+1 )
    nu <- nuMinusOne+1
    nuMinusOne  ̃ dexp(1/29.0)
    # Transform to original scale:
    beta[1:Nx] <- ( zbeta[1:Nx] / xsd[1:Nx] )*ysd
    beta0 <- zbeta0*ysd + ym - sum( zbeta[1:Nx] * xm[1:Nx] / xsd[1:Nx] )*ysd
    sigma <- zsigma*ysd
}

#### Jags-Ymet-XmetMulti-Mrobust-Example.R

In [1]:
# 책에는 없는 코드. 실습을 위해 data 폴더에 넣은 코드를 실행시키기 위한 작업
cur_dir = getwd()
setwd(sprintf("%s/%s", cur_dir, 'data'))

In [2]:
# Example for Jags-Ymet-XmetMulti-Mrobust.R 
#------------------------------------------------------------------------------- 
# Optional generic preliminaries:
#graphics.off() # This closes all of R's graphics windows.
#rm(list=ls())  # Careful! This clears all of R's memory!

In [3]:
#.............................................................................
# # Two predictors:
myData = read.csv( file="Guber1999data.csv" )

In [4]:
myData

,State,Spend,StuTeaRat,Salary,PrcntTake,SATV,SATM,SATT
1,Alabama,4.405,17.2,31.144,8,491,538,1029
2,Alaska,8.963,17.6,47.951,47,445,489,934
3,Arizona,4.778,19.3,32.175,27,448,496,944
4,Arkansas,4.459,17.1,28.934,6,482,523,1005
5,California,4.992,24,41.078,45,417,485,902
6,Colorado,5.443,18.4,34.571,29,462,518,980
7,Connecticut,8.817,14.4,50.045,81,431,477,908
8,Delaware,7.03,16.6,39.076,68,429,468,897
9,Florida,5.718,19.1,32.588,48,420,469,889
10,Georgia,5.193,16.3,32.291,65,406,448,854


In [5]:
summary(myData)

        State        Spend         StuTeaRat         Salary     
 Alabama   : 1   Min.   :3.656   Min.   :13.80   Min.   :25.99  
 Alaska    : 1   1st Qu.:4.882   1st Qu.:15.22   1st Qu.:30.98  
 Arizona   : 1   Median :5.768   Median :16.60   Median :33.29  
 Arkansas  : 1   Mean   :5.905   Mean   :16.86   Mean   :34.83  
 California: 1   3rd Qu.:6.434   3rd Qu.:17.57   3rd Qu.:38.55  
 Colorado  : 1   Max.   :9.774   Max.   :24.30   Max.   :50.05  
 (Other)   :44                                                  
   PrcntTake          SATV            SATM            SATT       
 Min.   : 4.00   Min.   :401.0   Min.   :443.0   Min.   : 844.0  
 1st Qu.: 9.00   1st Qu.:427.2   1st Qu.:474.8   1st Qu.: 897.2  
 Median :28.00   Median :448.0   Median :497.5   Median : 945.5  
 Mean   :35.24   Mean   :457.1   Mean   :508.8   Mean   : 965.9  
 3rd Qu.:63.00   3rd Qu.:490.2   3rd Qu.:539.5   3rd Qu.:1032.0  
 Max.   :81.00   Max.   :516.0   Max.   :592.0   Max.   :1107.0  
                  

In [6]:
yName = "SATT" ; xName = c("Spend","PrcntTake")
fileNameRoot = "Guber1999data-Jags-" 
numSavedSteps=15000 ; thinSteps=5

In [7]:
graphFileType = "png" 

In [8]:
#------------------------------------------------------------------------------- 
# Load the relevant model into R's working memory:
source("Jags-Ymet-XmetMulti-Mrobust.R")


*********************************************************************
Kruschke, J. K. (2015). Doing Bayesian Data Analysis, Second Edition:
A Tutorial with R, JAGS, and Stan. Academic Press / Elsevier.
*********************************************************************



Loading required package: coda


In [9]:
#------------------------------------------------------------------------------- 
# Generate the MCMC chain:
#startTime = proc.time()
mcmcCoda = genMCMC( data=myData , xName=xName , yName=yName , 
                    numSavedSteps=numSavedSteps , thinSteps=thinSteps , 
                    saveName=fileNameRoot )


CORRELATION MATRIX OF PREDICTORS:
           Spend PrcntTake
Spend     1.000     0.593
PrcntTake 0.593     1.000

Calling 3 simulations using the parallel method...
Following the progress of chain 1 (the program will wait for all chains
to finish before continuing):
Welcome to JAGS 3.4.0 on Wed Sep 16 20:42:43 2015
JAGS is free software and comes with ABSOLUTELY NO WARRANTY
Loading module: basemod: ok
Loading module: bugs: ok
. . Reading data file data.txt
. Compiling data graph
   Resolving undeclared variables
   Allocating nodes
   Initializing
   Reading data back into data table
Compiling model graph
   Resolving undeclared variables
   Allocating nodes
   Graph Size: 535
. Reading parameter file inits1.txt
. Initializing model
. Adapting 500
-------------------------------------------------| 500
++++++++++++++++++++++++++++++++++++++++++++++++++ 100%
Adaptation successful
. Updating 1000
-------------------------------------------------| 1000
************************************

In [10]:
#------------------------------------------------------------------------------- 
# Display diagnostics of chain, for specified parameters:
parameterNames = varnames(mcmcCoda) # get all parameter names
for ( parName in parameterNames ) {
  diagMCMC( codaObject=mcmcCoda , parName=parName , 
            saveName=fileNameRoot , saveType=graphFileType )
}

In [11]:
# 창으로 뜨는 것들을 없앤다.
graphics.off()

In [12]:
#------------------------------------------------------------------------------- 
# Get summary statistics of chain:
summaryInfo = smryMCMC( mcmcCoda , 
                        saveName=fileNameRoot )
show(summaryInfo)

                   Mean        Median          Mode     ESS HDImass      HDIlow
CHAIN       2.000000000   2.000000000  1.997413e+00     1.5    0.95   1.0000000
beta0     991.492221267 991.336500000  9.899081e+02 13808.0    0.95 948.3840000
beta[1]    12.832703606  12.858900000  1.273112e+01 13153.9    0.95   4.5361400
beta[2]    -2.878617251  -2.879090000 -2.881414e+00 13227.0    0.95  -3.3193300
sigma      31.530577473  31.329000000  3.106441e+01 13692.3    0.95  23.9116000
zbeta0     -0.001200131  -0.001465985 -9.973416e-05 15000.0    0.95  -0.1229360
zbeta[1]    0.233739136   0.234216000  2.318889e-01 13153.9    0.95   0.0826227
zbeta[2]   -1.029646875  -1.029815000 -1.030645e+00 13227.0    0.95  -1.1872800
zsigma      0.421415965   0.418722000  4.151860e-01 13692.3    0.95   0.3195860
nu         33.397469008  24.824800000  1.103249e+01 10801.1    0.95   1.8336700
log10(nu)   1.375887230   1.394885757  1.485261e+00 12329.1    0.95   0.6664712
              HDIhigh CompVal PcntGtComp

In [13]:
# Display posterior information:
plotMCMC( mcmcCoda , data=myData , xName=xName , yName=yName , 
          pairsPlot=TRUE , showCurve=FALSE ,
          saveName=fileNameRoot , saveType=graphFileType )

In [14]:
# 창으로 뜨는 것들을 없앤다.
graphics.off()

## 18.1.3 The posterior distribution

<img src="figures/fig18.5.1.png" width=600 />

<img src="figures/fig18.5.2.png" width=600 />

### proportion of variance accounted for

#### frequentist

* In least-squares regression, the overall variance in y is algebraically decomposed into the variance of the linearly predicted values and the residual variance:

<img src="figures/cap18.new.1.png" width=600 />

* The proportion of variance accounted for is

<img src="figures/cap18.new.2.png" />

#### bayesian

* Bayesian analysis no such decomposition of variance occurs
* But for people familiar with least-squares notions who crave a statistic analogous to R^2 , we
can compute a surrogate. At each step in the MCMC chain, a credible value of R^2 is computed as

<img src="figures/cap18.new.3.png" />

where

<img src="figures/cap18.new.4.png" />

is the standardized regression coefficient for the jth predictor at that step in the MCMC chain,

and

<img src="figures/cap18.new.5.png" />

is the correlation of the predicted values,

<img src="figures/cap18.new.6.png" />

with the jth predictor values,

<img src="figures/cap18.new.7.png" />

## 18.1.4 Redundant predictors

<img src="figures/fig18.6.1.png" width=600 />

<img src="figures/fig18.6.2.png" width=600 />

<img src="figures/fig18.7.1.png" width=600 />

<img src="figures/fig18.7.2.png" width=600 />

<img src="figures/cap18.1.png"  />

## 18.1.5 Informative priors, sparse data, and correlated predictors

#### Informed priors 

* The examples in this book tend to use mildly informed priors (e.g., using information about the rough magnitude and range of the data). But a benefit of Bayesian analysis is the potential for cumulative scientific progress by using priors that have been informed from previous research.

#### sparse data

* Informed priors can be especially useful when the amount of data is small compared to the parameter space.
* A strongly informed prior essentially reduces the scope of the credible parameter space, so that a small amount of new data implies a narrow zone of credible parameter values.

#### correlated predictors

* In the context of multiple linear regression, sparse data can lead to usefully precise posteriors on regression coefficients if some of the regression coefficients have informed priors and the predictors are strongly correlated. 
* To understand this idea, it is important to remember that when predictors are correlated, their regression coefficients are also (anti-)correlated.
* For example, 
    - recall the SAT data from Figure 18.3 (p. 514) in which spending-per-pupil and percent-taking-the-exam are correlated. Consequently, the posterior estimates of the regression coefficients had a negative correlation, as shown in Figure 18.5 (p. 518). The correlation of credible regression coefficients implies that a strong belief about the value of one regression coefficient constrains the value of the other coefficient. 
    - That influence of one slope estimate on another can be used for inferential advantage when we have prior knowledge about one of the slopes. 

# 18.2. Multiplicative Interaction of Metric Predictors

* 18.2.1 An example

<img src="figures/eq18.2-4.png" width=600 />

<img src="figures/fig18.8.png" width=600 />

## 18.2.1 An example

we conceptualize the interaction term of Equation 18.2 as an additional additive predictor, like this:

<img src="figures/eq18.5.png" width=600 />

* We create the new variable x3 = x1x2 outside the model and then submit the new variable as if it were another additive predictor.
    - One benefit of this approach is that we do not have to create a new model, and it is easy, in cases of many predictors, to set up interaction variables for many different combinations of variables. 
    - Another key benefit is that we can examine the correlations of the single predictors with the interaction variables. 

* To illustrate some of the issues involved in interpreting the parameters of a model with interaction, consider again the SAT data from Figure 18.3. 
    - SAT score
    - the spending per pupil (Spend) 
    - the percentage of students who took the test (PrcntTake)
    - When no interaction term was included in the model, the posterior distribution looked like Figure 18.5, which indicated a positive influence of Spend and a negative influence of PrcntTake.
* We will include a multiplicative interaction of Spend and PrcntTake.

#### Jags-Ymet- XmetMulti-Mrobust-Example.R

In [ ]:
# Read in data:
myData = read.csv( file="Guber1999data.csv" )
# Append the new interaction variable:
myData = cbind( myData , SpendXPrcnt = myData[,"Spend"]  * myData[,"PrcntTake"] )
# Specify names of data columns to use in the analysis:
yName = "SATT" ; xName = c("Spend","PrcntTake","SpendXPrcnt")

In [ ]:
#### 수정된 버전 Jags-Ymet-XmetMulti-Mrobust-Example.R

In [1]:
# 책에는 없는 코드. 실습을 위해 data 폴더에 넣은 코드를 실행시키기 위한 작업
cur_dir = getwd()
setwd(sprintf("%s/%s", cur_dir, 'data'))

In [2]:
#.............................................................................
# # Two predictors:
myData = read.csv( file="Guber1999data.csv" )

In [3]:
# Append the new interaction variable:
myData = cbind( myData , SpendXPrcnt = myData[,"Spend"]  * myData[,"PrcntTake"] )
# Specify names of data columns to use in the analysis:
yName = "SATT" ; xName = c("Spend","PrcntTake","SpendXPrcnt")

In [4]:
myData

,State,Spend,StuTeaRat,Salary,PrcntTake,SATV,SATM,SATT,SpendXPrcnt
1,Alabama,4.405,17.2,31.144,8,491,538,1029,35.24
2,Alaska,8.963,17.6,47.951,47,445,489,934,421.261
3,Arizona,4.778,19.3,32.175,27,448,496,944,129.006
4,Arkansas,4.459,17.1,28.934,6,482,523,1005,26.754
5,California,4.992,24,41.078,45,417,485,902,224.64
6,Colorado,5.443,18.4,34.571,29,462,518,980,157.847
7,Connecticut,8.817,14.4,50.045,81,431,477,908,714.177
8,Delaware,7.03,16.6,39.076,68,429,468,897,478.04
9,Florida,5.718,19.1,32.588,48,420,469,889,274.464
10,Georgia,5.193,16.3,32.291,65,406,448,854,337.545


In [5]:
fileNameRoot = "Guber1999data-interaction-Jags-" 
numSavedSteps=15000 ; thinSteps=5

In [6]:
graphFileType = "png" 

In [7]:
#------------------------------------------------------------------------------- 
# Load the relevant model into R's working memory:
source("Jags-Ymet-XmetMulti-Mrobust.R")


*********************************************************************
Kruschke, J. K. (2015). Doing Bayesian Data Analysis, Second Edition:
A Tutorial with R, JAGS, and Stan. Academic Press / Elsevier.
*********************************************************************



Loading required package: coda


In [8]:
#------------------------------------------------------------------------------- 
# Generate the MCMC chain:
#startTime = proc.time()
mcmcCoda = genMCMC( data=myData , xName=xName , yName=yName , 
                    numSavedSteps=numSavedSteps , thinSteps=thinSteps , 
                    saveName=fileNameRoot )


CORRELATION MATRIX OF PREDICTORS:
             Spend PrcntTake SpendXPrcnt
Spend       1.000     0.593       0.775
PrcntTake   0.593     1.000       0.951
SpendXPrcnt 0.775     0.951       1.000

Calling 3 simulations using the parallel method...
Following the progress of chain 1 (the program will wait for all chains
to finish before continuing):
Welcome to JAGS 3.4.0 on Wed Oct  7 02:03:51 2015
JAGS is free software and comes with ABSOLUTELY NO WARRANTY
Loading module: basemod: ok
Loading module: bugs: ok
. . Reading data file data.txt
. Compiling data graph
   Resolving undeclared variables
   Allocating nodes
   Initializing
   Reading data back into data table
Compiling model graph
   Resolving undeclared variables
   Allocating nodes
   Graph Size: 638
. Reading parameter file inits1.txt
. Initializing model
. Adapting 500
-------------------------------------------------| 500
++++++++++++++++++++++++++++++++++++++++++++++++++ 100%
Adaptation successful
. Updating 1000
----------

In [9]:
#------------------------------------------------------------------------------- 
# Display diagnostics of chain, for specified parameters:
parameterNames = varnames(mcmcCoda) # get all parameter names
for ( parName in parameterNames ) {
  diagMCMC( codaObject=mcmcCoda , parName=parName , 
            saveName=fileNameRoot , saveType=graphFileType )
}

In [10]:
# 창으로 뜨는 것들을 없앤다.
graphics.off()

In [11]:
#------------------------------------------------------------------------------- 
# Get summary statistics of chain:
summaryInfo = smryMCMC( mcmcCoda , 
                        saveName=fileNameRoot )
show(summaryInfo)

                   Mean        Median          Mode     ESS HDImass      HDIlow
CHAIN      2.000000e+00    2.00000000  1.997413e+00     1.5    0.95   1.0000000
beta0      1.046328e+03 1046.55500000  1.045477e+03  1192.8    0.95 964.1770000
beta[1]    2.706973e+00    2.64031500  2.081181e+00  1215.5    0.95 -13.2015000
beta[2]   -4.064498e+00   -4.06793500 -4.089410e+00   898.6    0.95  -5.6597000
beta[3]    2.037744e-01    0.20437850  2.117125e-01   813.3    0.95  -0.0561615
sigma      3.104436e+01   30.81855000  3.040303e+01 13947.4    0.95  23.9590000
zbeta0    -1.564266e-03   -0.00124368  3.764121e-03 15000.0    0.95  -0.1236310
zbeta[1]   4.930570e-02    0.04809150  3.790802e-02  1215.5    0.95  -0.2404560
zbeta[2]  -1.453822e+00   -1.45505000 -1.462733e+00   898.6    0.95  -2.0244100
zbeta[3]   5.615321e-01    0.56319650  5.834098e-01   813.3    0.95  -0.1547620
zsigma     4.149175e-01    0.41189950  4.063463e-01 13947.4    0.95   0.3202200
nu         3.490538e+01   26.42730000  1

In [12]:
# Display posterior information:
plotMCMC( mcmcCoda , data=myData , xName=xName , yName=yName , 
          pairsPlot=TRUE , showCurve=FALSE ,
          saveName=fileNameRoot , saveType=graphFileType )

In [13]:
# 창으로 뜨는 것들을 없앤다.
graphics.off()

When the analysis is run, the first thing it does is display the correlations of the predictors:

<img src="figures/cap18.2.png" width=600 />

* We can see that the interaction variable is strongly correlated with both predictors. 
* Therefore, we know that 
    - there will be strong trade-offs among the regression coefficients, 
    - and the marginal distributions of single regression coefficients might be much wider than when there was no interaction included.

<img src="figures/fig18.9.1.png" width=600 />

* The marginal distribution for β3, also labeled as SpendXPrcnt, indicates that the modal value of the interaction coefficients is indeed positive, as we anticipated it could be. However, the 95% HDI includes zero, which indicates that we do not have very strong precision in the estimate of the magnitude of the interaction.

<img src="figures/fig18.9.2.png" width=600 />

# 18.3. Shrinkage of Regression Coefficients

<img src="figures/fig18.10.png" width=600 />

<img src="figures/fig18.11.1.png" width=600 />

<img src="figures/fig18.11.2.png" width=600 />

<img src="figures/fig18.12.1.png" width=600 />

<img src="figures/fig18.12.2.png" width=600 />

# 18.4. Variable Selection

* 18.4.1 Inclusion probability is strongly affected by vagueness of prior
* 18.4.2 Variable selection with hierarchical shrinkage
* 18.4.3 What to report and what to conclude
* 18.4.4 Caution: Computational methods
* 18.4.5 Caution: Interaction variables

<img src="figures/eq18.6.png" width=600 />

<img src="figures/fig18.13.png" width=600 />

## 18.4.1 Inclusion probability is strongly affected by vagueness of prior

<img src="figures/fig18.14.1.png" width=550 />

<img src="figures/fig18.14.2.png" width=600 />

## 18.4.2 Variable selection with hierarchical shrinkage

<img src="figures/cap18.3.png" width=600 />

## 18.4.3 What to report and what to conclude

<img src="figures/fig18.15.1.png" width=600 />
<img src="figures/fig18.15.2.png" width=600 />
<img src="figures/fig18.15.3.png" width=600 />

## 18.4.4 Caution: Computational methods

## 18.4.5 Caution: Interaction variables

# 참고자료